In [2]:
# default_exp parser
%load_ext autoreload
%autoreload 2    

Fonte: # https://github.com/fastai/fastai/blob/master/fastai/data/transforms.py#L26

In [3]:
#export
from collections import defaultdict
from fastcore.basics import uniqueify
from fastcore.utils import parallel
from fastcore.foundation import L
from rfpy.blocks import create_base_block, block_constructor
import pandas as pd
import numpy as np
import os
from pathlib import Path
from typing import *
from datetime import datetime

path_type = Union[str, Any]
bin_val = Union[int, bytes]
bytes_encoded = Union[int, bytes]
datetime_object = datetime

META = ['Block_Number', 'Latitude', 'Longitude', 'Altitude', 'Initial_Time',
     'Sample_Duration', 'Start_Frequency', 'Stop_Frequency', 'Vector_Length', 
     'Trace_Type', 'Antenna_Type', 'Equipement_ID']

COLS_B = ['Block_Number', 'Frequency(HZ)', 'Nivel(dBm)']

BLOCO_40 = ['latitude', 'longitude', 'altitude']

BLOCO_63 = ['datetime_stamp',
           'spent_time_microsecs', 'start_mega', 'stop_mega', 'data_points', 
           'processing', 'id_antenna']

def _get_files(p, fs, extensions=None):
    p = Path(p)
    res = [p/f for f in fs if not f.startswith('.')
           and ((not extensions) or f'.{f.split(".")[-1].lower()}' in extensions)]
    return res

def get_files(path, extensions=None, recurse=True, folders=None, followlinks=True):
    "Get all the files in `path` with optional `extensions`, optionally with `recurse`, only in `folders`, if specified."
    path = Path(path)
    folders=L(folders)
    if extensions is not None:
        extensions = set(uniqueify(extensions))
        extensions = {e.lower() for e in extensions}
    if recurse:
        res = []
        for i,(p,d,f) in enumerate(os.walk(path, followlinks=followlinks)): # returns (dirpath, dirnames, filenames)
            if len(folders) !=0 and i==0: d[:] = [o for o in d if o in folders]
            else:                         d[:] = [o for o in d if not o.startswith('.')]
            if len(folders) !=0 and i==0 and '.' not in folders: continue
            res += _get_files(p, f, extensions)
    else:
        f = [o.name for o in os.scandir(path) if o.is_file()]
        res = _get_files(path, f, extensions)
    return L(res)

C:\Users\rsilva\Miniconda3\envs\bin\lib\site-packages\fastprogress\fastprogress.py:102: UserWarning: Couldn't import ipywidgets properly, progress bar will use console behavior
  warn("Couldn't import ipywidgets properly, progress bar will use console behavior")


In [4]:
#export
def binary_file_generator(bin_file: path_type, marker: bytes = b'\x00UUUU', block_size: int = 4096) -> Iterator[bytes]:
    """
    str, bytes, int > bytes
    :param bin_file: arquivo binario que contém os dados
    :param marker: separador de blocos
    :param block_size: tamanho em bytes que é "lido" por vez no arquivo, evitando problemas de memória
    :return: bloco em formato binario
    Gerador que fornece a partir de de um arquivo binário, um bloco binário por vez.

    """
    with open(bin_file, mode='rb') as bfile:
        # O primeiro bloco do arquivo é o cabeçalho e tem 36 bytes de tamanho.
        yield bfile.read(36)
        # As demais partes podem prosseguir normalmente
        current = b''
        while True:
            block = bfile.read(block_size)
            if not block:  # end-of-file
                # yield current
                return None
            current += block
            while True:
                markerpos = current.find(marker)
                if markerpos < 0:
                    break
                yield current[:markerpos]
                current = current[markerpos + len(marker):]

A função a seguir mapeia o arquivo `.bin` nos devidos blocos

In [5]:
#export
def file2block(file: Union[str, Path])->Mapping[int,L]:
    """Receives a path to a bin file and returns a defaultdict with unique block types as keys and a list of the Class Blocks as values
        :param file: A string or pathlib.Path like path to a `.bin`file generated by CFRS - Logger
        :return: A Dictionary with block types as keys and a list of the Class Blocks available as values
    """
    map_block = defaultdict(L)
    for bloco in L(binary_file_generator(file)).map(create_base_block):
        btype = bloco.type
        map_block[btype].append(block_constructor(btype, bloco))
    return map_block

In [6]:
#export
def _extract_level(bloco: Any):
    return np.expand_dims(bloco.block_data, 0) # reshape((-1, 1))
     
def export_bin_level(map_block: Mapping[int,L], pivoted: bool = True)->pd.DataFrame:
    """Receives a mapping `map_block` and returns the Matrix with the Levels as values, Frequencies as columns and Block Number as index.
       :param pivoted: If False, optionally returns an unpivoted version of the Matrix
    """
    spectrum_blocks = map_block[63]
    assert len(spectrum_blocks), f"The spectrum block list is empty"
    levels = np.concatenate(parallel(_extract_level, spectrum_blocks, n_workers=8, progress=False))
    frequencies = getattr(spectrum_blocks[0], 'list_freq_mega')
    pivot = pd.DataFrame(levels, columns=frequencies)
    if not pivoted:
        unpivot = pivot.melt(var_name="Frequency(HZ)", value_name="Nivel(dBm)")
        unpivot['Block_Number'] = levels.shape[0] * list(range(levels.shape[1]))
        unpivot.sort_values('Block_Number', inplace=True).reset_index(drop=True, inplace=True)
        return unpivot
    return pivot

In [7]:
#export
def export_bin_meta(map_block: Mapping[int,L])->pd.DataFrame:
    """Receives a Mapping with the different `. bin` Blocks and extracts the metadata listed in `META` in a dataframe format
    """
    assert len(map_block[40]) == len(map_block[63]), "Check if your Binary File is of type 63. GPS Block 40 length != Spectrum Block 63"
    n_data = len(map_block[63])
    output = {}
    output['Block_Number'] = L(range(n_data)) 
    
    for attr in BLOCO_40:
        output[attr] = map_block[40].attrgot(attr)
    for attr in BLOCO_63:
        output[attr] = map_block[63].attrgot(attr)            
    
    output['Equipement_ID'] = L(getattr(map_block[21][0], 'hostname')) * n_data    
    df = pd.DataFrame(output)    
    df.columns = META
    return df

In [8]:
#hide
from nbdev.export import notebook2script; notebook2script()

Converted 00_main.ipynb.
Converted 01_parser.ipynb.
Converted 02_utils.ipynb.
Converted 03_blocks.ipynb.
Converted index.ipynb.
